Using the McBride and Wark biographies of Acker to construct a detailed composite biography of Acker's life up to the publication of Black Tarantula. Using this fabricated composite biography, we can distill it into data points based around distinct "events" in Acker's life, with a description of the event as well as mention of all actors involved, and finally which sentences from the original biographies were used to inform and define an event.

This can be visualised as a network exploration in itself, but it can also be compared against extracts from Black Tarantula to identify biographical influences on the texts, in the same way more apparent literary sources are identified.

In [12]:
import xml.etree.ElementTree as ET
import os
import json
import nltk
import re

# Create a dictionary to store the chapters for each XML file
chapters_dict = {}

# Parse XML
tree = ET.parse('xml/bio/mcbride.xml')
root = tree.getroot()

# Iterate over all divs with type chapter
for div in root.findall('.//{http://www.tei-c.org/ns/1.0}div[@type="chapter"]'):
    string = ""
    result = []
    # Iterate over all paragraphs in the respective chapter
    for p in div.findall('.//{http://www.tei-c.org/ns/1.0}p'):
        # Removing all empty nodes
        if p.text is None:
            p.text = ''
        # Incorporate the text contents of the <quote> elements into the <p> text
        for quote in p.findall('.//{http://www.tei-c.org/ns/1.0}quote'):
            if quote.text is not None:
                p.text += quote.text
            for lg in quote.findall('.//{http://www.tei-c.org/ns/1.0}lg'):
                if lg.text is not None:
                    p.text += lg.text
                for l in lg.findall('.//{http://www.tei-c.org/ns/1.0}l'):
                    if l.text is not None:
                        p.text += l.text
        string += str(p.text) + " "
    result = "".join(string)

    # Add the chapter with only its respective text to the dictionary
    chapters_dict[div.attrib['n']] = result

# Write the dictionary to a JSON file and save to the xml folder
with open('xml/bio/mcbride_chapters.json', 'w') as fp:
    json.dump(chapters_dict, fp, indent=2)

In [13]:
import torch
from transformers import pipeline
import json

summarizer = pipeline(
    "summarization",
    "pszemraj/long-t5-tglobal-base-16384-book-summary",
    device=0 if torch.cuda.is_available() else -1,
)

# Load the JSON file with the chapter texts
with open('xml/bio/mcbride_chapters.json', 'r') as fp:
    chapters_dict = json.load(fp)

# Iterate through the chapter texts and summarize each one
for chapter, text in chapters_dict.items():
    result = summarizer(text)
    summary = result[0]["summary_text"]
    # Add the summary to the dictionary with the corresponding chapter
    chapters_dict[chapter] = {"text": text, "summary": summary}

# Write the updated dictionary to the JSON file
with open('xml/bio/mcbride_chapters.json', 'w') as fp:
    json.dump(chapters_dict, fp, indent=2)


/Users/mderdun/anaconda3/envs/ackerText/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Users/mderdun/anaconda3/envs/ackerText/lib/python3.11/site-packages/transformers/models/longt5/modeling_longt5.py:171: UserWarning: An output with one or more elements was resized since it had shape [2717], which does not match the required output shape [1, 2717]. This behavior is deprecated, and in a future PyTorch release outputs will not be resized unless they have zero elements. You can explicitly reuse an out tensor t by resizing it, inplace, to zero elements with t.resize_(0). (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/aten/src/ATen/native/Resize.cpp:35.)
  true_block_ends = torch.logical_and(block_ends, block_ids >= 0)
/Users/mderdun/anaconda3/envs/ackerText/lib/python3.11/site-packag